# Lesson 24 - CIFAR-10 Cats and Dogs Round 2
Austin Derrow-Pinion CM 208

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import scale
import time

np.random.seed(0)

In [2]:
# load CIFAR-10 Cats and Dogs split data set
CIFAR10 = np.load('./Data/CIFAR-10_cats_dogs_split.npz')

# label names
label_names = CIFAR10['label_names']

# training data
images_train = CIFAR10['images_train']
labels_train = CIFAR10['labels_train']

# validation data
images_valid = CIFAR10['images_valid']
labels_valid = CIFAR10['labels_valid']

# test data
images_test = CIFAR10['images_test']
labels_test = CIFAR10['labels_test']

# labels and shapes of data
print(label_names)
print('images_train shape:', images_train.shape)
print('labels_train shape:', labels_train.shape)
print('images_valid shape:', images_valid.shape)
print('labels_valid shape:', labels_valid.shape)
print('images_test shape:', images_test.shape)
print('labels_test shape:', labels_test.shape)

['airplane' 'automobile' 'bird' 'cat' 'deer' 'dog' 'frog' 'horse' 'ship'
 'truck']
images_train shape: (6000, 32, 32, 3)
labels_train shape: (6000,)
images_valid shape: (2000, 32, 32, 3)
labels_valid shape: (2000,)
images_test shape: (2000, 32, 32, 3)
labels_test shape: (2000,)


In [3]:
# scale data
image_size = 32*32*3

images_train_s = scale(np.reshape(np.array(images_train, dtype='float32'), [-1, image_size]), axis=1)
images_valid_s = scale(np.reshape(np.array(images_valid, dtype='float32'), [-1, image_size]), axis=1)
images_test_s = scale(np.reshape(np.array(images_test, dtype='float32'), [-1, image_size]), axis=1)

/opt/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/opt/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:184: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [4]:
# Tensorflow computational graph
x = tf.placeholder(tf.float32, shape=[None, image_size])
y = tf.placeholder(tf.float32, shape=[None])
y_one_hot = tf.one_hot(tf.to_int32(y), depth=10, on_value=1.0, off_value=0.0, dtype=tf.float32)

W = tf.Variable(tf.truncated_normal([image_size, 10], dtype=tf.float32, seed=0, stddev=0.1))
b = tf.Variable(tf.truncated_normal([10], dtype=tf.float32, seed=0, stddev=0.1))

logits = tf.matmul(x, W) + b

CE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y_one_hot))
optimizer = tf.train.AdamOptimizer().minimize(CE)

y_pred = tf.argmax(tf.nn.softmax(logits), dimension=1)
accuracy = 1.0 - tf.contrib.metrics.accuracy(tf.to_int32(y_pred), tf.to_int32(y))

In [5]:
def minibatches(x, y, batchsize):
    assert x.shape[0] == y.shape[0]
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    
    for i in range(0, x.shape[0] - batchsize + 1, batchsize):
        excerpt = indices[i:i + batchsize]
        yield np.array(x[excerpt], dtype='float32'), np.array(y[excerpt], dtype='int32')
    
    leftOver = (x.shape[0] - batchsize) % batchsize
    if leftOver != 0:
        excerpt = indices[len(indices) - leftOver:]
        yield np.array(x[excerpt], dtype='float32'), np.array(y[excerpt], dtype='int32')

In [6]:
init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    
    num_epochs = 0
    EPOCHS = 1000
    batch_size = 1000
    starting_time = time.time()
    
    while num_epochs < EPOCHS:
        
        for batch in minibatches(images_train_s, labels_train, batch_size):
            batch_x, batch_y = batch
            _ = sess.run([optimizer], feed_dict={x: batch_x, y: batch_y})

        
        num_epochs += 1
        if (num_epochs % 100 == 0):
            training_CE, training_error = sess.run([CE, accuracy],
                                                   feed_dict={x: images_train_s,
                                                              y: labels_train})
            validation_CE, validation_error = sess.run([CE, accuracy],
                                                      feed_dict={x: images_valid_s,
                                                                 y: labels_valid})
            print("EPOCH = {:d}".format(num_epochs))
            print("    training cross-entropy = {:.5f}".format(training_CE))
            print("  validation cross-entropy = {:.5f}".format(validation_CE))
            print("       training error rate = {:.3f}".format(training_error))
            print("     validation error rate = {:.3f}".format(validation_error))
            print("      total time (minutes) = {:.1f}".format((time.time() - starting_time) / 60.0))

EPOCH = 100
    training cross-entropy = 0.84290
  validation cross-entropy = 1.19754
       training error rate = 0.332
     validation error rate = 0.489
      total time (minutes) = 0.3
EPOCH = 200
    training cross-entropy = 0.68290
  validation cross-entropy = 1.07068
       training error rate = 0.297
     validation error rate = 0.476
      total time (minutes) = 0.6
EPOCH = 300
    training cross-entropy = 0.59538
  validation cross-entropy = 0.98755
       training error rate = 0.263
     validation error rate = 0.456
      total time (minutes) = 0.8
EPOCH = 400
    training cross-entropy = 0.56376
  validation cross-entropy = 0.95238
       training error rate = 0.266
     validation error rate = 0.454
      total time (minutes) = 1.1
EPOCH = 500
    training cross-entropy = 0.53360
  validation cross-entropy = 0.92521
       training error rate = 0.248
     validation error rate = 0.453
      total time (minutes) = 1.3
EPOCH = 600
    training cross-entropy = 0.51747
  vali